In [ ]:
# Install this package to use Colab's GPU for training
!apt install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6

In [ ]:
import os
import zipfile
from zipfile import ZipFile
import matplotlib.pyplot as plt
import tensorflow as tf
import shutil
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
zip_file_path =  '/content/drive/MyDrive/PCB_Detection/VOC_PCB.zip'
print(zip_file_path)

In [ ]:
# ## Extracting the content 

with ZipFile(zip_file_path, 'r') as zip:
  # print(zip.printdir())
  zip.extractall('/content/drive/MyDrive/PCB_Detection')

In [ ]:
main_path = '/content/drive/MyDrive/PCB_Detection/VOC_PCB'

In [ ]:
main_images_path = '/content/drive/MyDrive/PCB_Detection/VOC_PCB/JPEGImages'

In [ ]:
categoris = ['missing_hole','mouse_bite','open_circuit','short','spurious_copper','spur']

In [ ]:
### creating dataset folder 
dataset_path = main_path + '/dataset'
try:
  os.makedirs(dataset_path)
except:
  print('Directory already exists!')

In [ ]:
train_path = dataset_path + '/train'
test_path = dataset_path + '/test'
validation_path = dataset_path + '/validation'
try:
  os.makedirs(train_path)
  os.makedirs(test_path)
  os.makedirs(validation_path)
except:
  print('Directories already exist!')

In [ ]:
def create_dataset(file, target_path, main_images_path=main_images_path):
  files = open(file, 'r').read().split('\n')
  all_images = os.listdir(main_images_path)
  # print(all_images[0])
  counter = 1
  for file in files:
    for category in categoris:
      if category in file:
        try:
          target_path1 = target_path + '/' + category
          os.makedirs(target_path1)
        except:
          print('already_exists')
        if file+".jpg" in all_images:
          source = main_images_path + '/' + file + '.jpg'
          target = target_path1 + '/' + file + '.jpg'
          # print(source, target)
          # print('*'*100)
          shutil.copyfile(source, target)
          os.rename(target, target_path1 + '/' + category + '_' + str(counter) +'.jpg')
          counter += 1
  # print(len(os.listdir(target_path)))


        





In [ ]:
train_files_names = '/content/drive/MyDrive/PCB_Detection/VOC_PCB/ImageSets/Main/train.txt'
create_dataset(train_files_names, train_path)

In [ ]:
validation_files_name = '/content/drive/MyDrive/PCB_Detection/VOC_PCB/ImageSets/Main/val.txt'
create_dataset(validation_files_name, validation_path) 

In [ ]:
test_files_name = '/content/drive/MyDrive/PCB_Detection/VOC_PCB/ImageSets/Main/test.txt'
create_dataset(test_files_name, test_path)

In [ ]:
# import re
# sub_directories = os.listdir(train_path)
# # print(os.listdir(train_path+'/'+sub_directories[0]))
# test = 'short_1111.jpg'
# print(test)
# test = re.sub(r'[0-9]+', str(0), test)
# print(test)
# # os.listdir(train_path+'/'+sub_directories[0])[0]

# def renaming_images(path):
#   sub_directories = os.listdir(path)
#   for dir in sub_directories:
#     counter = 0
#     files = os.listdir(sub_directories)
#     for file in files:
#       file

In [ ]:
train_path_generator = ImageDataGenerator(rescale=1.0/255.0,
                                          rotation_range=40,
                                          horizontal_flip=True,
                                          shear_range=0.2,
                                          height_shift_range=0.2,
                                          width_shift_range=0.2,
                                          validation_split=0.2,
                                          zoom_range=0.2)

train_dataset = train_path_generator.flow_from_directory(
                                                         train_path,
                                                         class_mode='categorical',
                                                         batch_size=20,
                                                         target_size=(600, 600)
                                                        )

In [ ]:
val_gen = ImageDataGenerator(rescale=1.0/255.0)
val_dataset = val_gen.flow_from_directory(
                                          validation_path,
                                          class_mode='categorical',
                                          batch_size=20,
                                          target_size=(600, 600)
                                        )

In [ ]:
### model building

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, (3, 3), input_shape=(600, 600, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(6, activation='softmax')
])

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])
history = model.fit(train_dataset,
          validation_data=val_dataset,
          epochs=10)

In [ ]:
## Plotting accuracy
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.legend()
plt.ylabel('Accuracy')
plt.title('Accuracy')
plt.show()

In [ ]:
## Plotting loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.legend()
plt.ylabel('loss')
plt.title('Loss')
plt.show()

In [ ]:
### Changing optimizer
### model building

model_do = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, (3, 3), input_shape=(150, 150, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(6, activation='softmax')
])


model_do.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.RMSprop(),
              metrics=['accuracy'])
history_rms_do = model_do.fit(train_dataset,
          validation_data=val_dataset,
          epochs=10)

In [ ]:
### Transfer Learning
from tensorflow.keras.applications import ResNet50

In [ ]:
feature_extractor = ResNet50(weights='imagenet', 
                             input_shape=(600, 600, 3),
                             include_top=False)

In [ ]:
feature_extractor.trainable = False


In [ ]:
for layer in feature_extractor.layers[:-2]:
  layer.trainable = True

In [ ]:
input_ = tf.keras.Input(shape=(600, 600, 3))
x = feature_extractor(input_, training=True)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
output_ = tf.keras.layers.Dense(6, activation='softmax')(x)
model_ResNet = tf.keras.Model(input_, output_)
model_ResNet.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model_ResNet.summary()

In [ ]:
history_resnet = model_ResNet.fit(train_dataset,
          validation_data=val_dataset,
          epochs=10)

In [ ]:
epochs = len(history_resnet.history['accuracy'])
plt.plot(range(epochs), history_resnet.history['accuracy'], label='Trainig Accuracy')
plt.plot(range(epochs), history_resnet.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Vs Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(range(epochs), history_resnet.history['loss'], label='Trainig loss')
plt.plot(range(epochs), history_resnet.history['val_loss'], label='Validation loss')
plt.title('Training loss Vs Val loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
### Getting VGG16 weights
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /content/drive/MyDrive/PCB_Detection/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers

In [ ]:
local_weight_files = '/content/drive/MyDrive/PCB_Detection/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
# Initialize the base model.
# Set the input shape and remove the dense layers.
pre_trained_model = InceptionV3(input_shape = (600, 600, 3), 
                                include_top = False, 
                                weights = None)

# Load the pre-trained weights you downloaded.
pre_trained_model.load_weights(local_weight_files)

# Freeze the weights of the layers.
for layer in pre_trained_model.layers:
  layer.trainable = False

In [ ]:
pre_trained_model.summary()

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')
last_layer_output = last_layer.output

In [ ]:
x = layers.Flatten()(last_layer_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(6, activation='softmax')(x)
model_VGG16 = Model(pre_trained_model.input, x)
model_VGG16.summary()

In [ ]:
from tensorflow.python import metrics
model_VGG16.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                    optimizer=tf.keras.optimizers.SGD(lr=1e-4),
                    metrics=['accuracy'])
history = model_VGG16.fit(
            train_dataset,
            validation_data = val_dataset,
            steps_per_epoch = 100,
            epochs = 10,
            validation_steps = 50)

In [ ]:
epochs = len(history.history['accuracy'])
plt.plot(range(epochs), history.history['accuracy'], label='Trainig Accuracy')
plt.plot(range(epochs), history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Vs Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(range(epochs), history.history['loss'], label='Trainig loss')
plt.plot(range(epochs), history.history['val_loss'], label='Validation loss')
plt.title('Training loss Vs Val loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()